In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
print("done")

done


In [2]:
# !pip install autogluon

In [3]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 53.5 MB/s e

In [4]:
!pip install autogluon.tabular[all,tabpfn]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 4.0 MB/s eta 0:00:00


In [5]:
path='../input/playground-series-s3e18/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
origin = pd.read_csv('/kaggle/input/ec-mixed-class/mixed_desc.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [6]:
len(train)

14838

In [7]:
train=train.drop("id",axis=1)
test=test.drop("id",axis=1)

In [8]:
columns_to_keep = [
    "BertzCT", "Chi1", "Chi1n", "Chi1v", "Chi2n", "Chi2v", "Chi3v", "Chi4n", 
    "EState_VSA1", "EState_VSA2", "ExactMolWt", "FpDensityMorgan1", "FpDensityMorgan2",
    "FpDensityMorgan3", "HallKierAlpha", "HeavyAtomMolWt", "Kappa3", "MaxAbsEStateIndex", 
    "MinEStateIndex", "NumHeteroatoms", "PEOE_VSA10", "PEOE_VSA14", "PEOE_VSA6", 
    "PEOE_VSA7", "PEOE_VSA8", "SMR_VSA10", "SMR_VSA5", "SlogP_VSA3", "VSA_EState9", 
    "fr_COO", "fr_COO2", "EC1_EC2_EC3_EC4_EC5_EC6"
]
origin = origin[columns_to_keep]

In [9]:
origin = origin.copy()
origin["EC1"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[0].astype('uint8')
origin["EC2"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[2].astype('uint8')
origin["EC3"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[4].astype('uint8')
origin["EC4"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[6].astype('uint8')
origin["EC5"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[8].astype('uint8')
origin["EC6"] = origin["EC1_EC2_EC3_EC4_EC5_EC6"].str[10].astype('uint8')
origin = origin.drop(columns=['EC1_EC2_EC3_EC4_EC5_EC6'])

In [10]:
cols = ['FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3']

for df_name, df in zip(["train", "origin"], [train, origin]):
    print(f"In {df_name}:")
    for col in cols:
        count = (df[col] == -666).sum()
        print(f"Number of -666 in {col}: {count}")

In train:
Number of -666 in FpDensityMorgan1: 1
Number of -666 in FpDensityMorgan2: 1
Number of -666 in FpDensityMorgan3: 1
In origin:
Number of -666 in FpDensityMorgan1: 1
Number of -666 in FpDensityMorgan2: 1
Number of -666 in FpDensityMorgan3: 1


In [11]:
#del outlier -666
cols = ['FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3']

for df_name in ["train", "origin"]:
    exec(f'{df_name} = {df_name}[~{df_name}[cols].isin([-666]).any(axis=1)].reset_index(drop=True)')
    exec(f'{df_name} = {df_name}[~{df_name}[cols].isin([-14.4584615]).any(axis=1)].reset_index(drop=True)')

In [12]:
columns_to_convert_uint8 = ['NumHeteroatoms', 'fr_COO', 'fr_COO2']
columns_to_convert_bool = ['EC1', 'EC2', 'EC3', 'EC4', 'EC5', 'EC6']
for column in columns_to_convert_uint8:
    train[column] = train[column].astype('uint8')
    test[column] = test[column].astype('uint8')
    origin[column]= origin[column].astype('uint8')
for column in columns_to_convert_bool:
    train[column] = train[column].astype('uint8')
    origin[column]= origin[column].astype('uint8')

In [13]:
train = pd.concat([train, origin])
len_before = len(train)


train = train.drop_duplicates().reset_index(drop=True)
len_after = len(train)

print("Number of duplicate rows removed:", len_before - len_after)

Number of duplicate rows removed: 36


In [14]:
import tabpfn

In [15]:
# hyperparameters={
#     'TABPFN': {},
#     'NN_TORCH': {},
#     'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
#     'CAT': {},
#     'XGB': {},
#     'FASTAI': {},
#     'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
#     'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
#     'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
# }

In [16]:
# hyperparameters={
#     'TABPFN': [{'N_ensemble_configurations': 5}],
#     'XT': [{'min_samples_leaf': 1, 'max_leaf_nodes': 15000, 'max_features': 0.5, 'ag_args': {'name_suffix': '_r19', 'priority': 20}}],
#     'RF': [{'min_samples_leaf': 5, 'max_leaf_nodes': 50000, 'max_features': 0.5, 'ag_args': {'name_suffix': '_r5', 'priority': 19}}],
#     'GBM': [{'extra_trees': False, 'feature_fraction': 0.7248284762542815, 'learning_rate': 0.07947286942946127, 'min_data_in_leaf': 50, 'num_leaves': 89, 'ag_args': {'name_suffix': '_r158', 'priority': 18}}, {'extra_trees': True, 'feature_fraction': 0.7832570544199176, 'learning_rate': 0.021720607471727896, 'min_data_in_leaf': 3, 'num_leaves': 21, 'ag_args': {'name_suffix': '_r118', 'priority': 17}}, {'extra_trees': True, 'feature_fraction': 0.7113010892989156, 'learning_rate': 0.012535427424259274, 'min_data_in_leaf': 16, 'num_leaves': 48, 'ag_args': {'name_suffix': '_r97', 'priority': 16}}, {'extra_trees': True, 'feature_fraction': 0.45555769907110816, 'learning_rate': 0.009591347321206594, 'min_data_in_leaf': 50, 'num_leaves': 110, 'ag_args': {'name_suffix': '_r71', 'priority': 15}}, {'extra_trees': False, 'feature_fraction': 0.40979710161022476, 'learning_rate': 0.008708890211023034, 'min_data_in_leaf': 3, 'num_leaves': 80, 'ag_args': {'name_suffix': '_r111', 'priority': 14}}],
#     'FASTAI': [{'bs': 1024, 'emb_drop': 0.6167722379778131, 'epochs': 44, 'layers': [200, 100, 50], 'lr': 0.053440377855629266, 'ps': 0.48477211305443607, 'ag_args': {'name_suffix': '_r25', 'priority': 13}}, {'bs': 1024, 'emb_drop': 0.6046989241462619, 'epochs': 48, 'layers': [200, 100, 50], 'lr': 0.00775309042164966, 'ps': 0.09244767444160731, 'ag_args': {'name_suffix': '_r51', 'priority': 12}}, {'bs': 512, 'emb_drop': 0.6557225316526186, 'epochs': 49, 'layers': [200, 100], 'lr': 0.023627682025564638, 'ps': 0.519566584552178, 'ag_args': {'name_suffix': '_r82', 'priority': 11}}, {'bs': 2048, 'emb_drop': 0.4066210919034579, 'epochs': 43, 'layers': [400, 200], 'lr': 0.0029598312717673434, 'ps': 0.4378695797438974, 'ag_args': {'name_suffix': '_r121', 'priority': 10}}, {'bs': 128, 'emb_drop': 0.44339037504795686, 'epochs': 31, 'layers': [400, 200, 100], 'lr': 0.008615195908919904, 'ps': 0.19220253419114286, 'ag_args': {'name_suffix': '_r145', 'priority': 9}}, {'bs': 128, 'emb_drop': 0.12106594798980945, 'epochs': 38, 'layers': [200, 100, 50], 'lr': 0.037991970245029975, 'ps': 0.33120008492595093, 'ag_args': {'name_suffix': '_r173', 'priority': 8}}, {'bs': 128, 'emb_drop': 0.4599138419358, 'epochs': 47, 'layers': [200, 100], 'lr': 0.03888383281136287, 'ps': 0.28193673177122863, 'ag_args': {'name_suffix': '_r128', 'priority': 7}}],
#     'CAT': [{'depth': 5, 'l2_leaf_reg': 4.774992314058497, 'learning_rate': 0.038551267822920274, 'ag_args': {'name_suffix': '_r16', 'priority': 6}}, {'depth': 4, 'l2_leaf_reg': 1.9950125740798321, 'learning_rate': 0.028091050379971633, 'ag_args': {'name_suffix': '_r42', 'priority': 5}}, {'depth': 6, 'l2_leaf_reg': 1.8298803017644376, 'learning_rate': 0.017844259810823604, 'ag_args': {'name_suffix': '_r93', 'priority': 4}}, {'depth': 7, 'l2_leaf_reg': 4.81099604606794, 'learning_rate': 0.019085060180573103, 'ag_args': {'name_suffix': '_r44', 'priority': 3}}]
# }


In [17]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label,problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data,tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
#             predictor.fit(train_data=train_data,hyperparameters=hyperparameters,tuning_data=tuning_data, **kwargs)
            predictor.fit(train_data=train_data,tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [18]:
def create_features(df):
    
    new_features = {
        'BertzCT_MaxAbsEStateIndex_Ratio': df['BertzCT'] / (df['MaxAbsEStateIndex'] + 1e-12),
        'BertzCT_ExactMolWt_Product': df['BertzCT'] * df['ExactMolWt'],
        'NumHeteroatoms_FpDensityMorgan1_Ratio': df['NumHeteroatoms'] / (df['FpDensityMorgan1'] + 1e-12),
        'VSA_EState9_EState_VSA1_Ratio': df['VSA_EState9'] / (df['EState_VSA1'] + 1e-12),
        'PEOE_VSA10_SMR_VSA5_Ratio': df['PEOE_VSA10'] / (df['SMR_VSA5'] + 1e-12),
        'Chi1v_ExactMolWt_Product': df['Chi1v'] * df['ExactMolWt'],
        'Chi2v_ExactMolWt_Product': df['Chi2v'] * df['ExactMolWt'],
        'Chi3v_ExactMolWt_Product': df['Chi3v'] * df['ExactMolWt'],
        'EState_VSA1_NumHeteroatoms_Product': df['EState_VSA1'] * df['NumHeteroatoms'],
        'PEOE_VSA10_Chi1_Ratio': df['PEOE_VSA10'] / (df['Chi1'] + 1e-12),
        'MaxAbsEStateIndex_NumHeteroatoms_Ratio': df['MaxAbsEStateIndex'] / (df['NumHeteroatoms'] + 1e-12),
        'BertzCT_Chi1_Ratio': df['BertzCT'] / (df['Chi1'] + 1e-12),
    }
    
    df = df.assign(**new_features)
    new_cols = list(new_features.keys())
    
    return df, new_cols
train,_=create_features(train)
test,_=create_features(test)

In [19]:
train = TabularDataset(train)
test = TabularDataset(test)

In [20]:
train = train.drop(columns=['EC6', 'EC3', 'EC4', 'EC5'])

In [21]:
excluded_model_types = ['KNN', 'NN_TORCH']
labels = ['EC1','EC2']
problem_types = ['binary','binary']
eval_metrics = ['roc_auc','roc_auc']
multi_predictor = MultilabelPredictor(labels=labels,problem_types=problem_types,sample_weight='auto_weight', eval_metrics=eval_metrics)
# multi_predictor.fit(train, presets="experimental_zeroshot_hpo_hybrid",fit_weighted_ensemble = True,excluded_model_types=excluded_model_types,time_limit=3600*6)
multi_predictor.fit(train,presets="experimental_best_quality",fit_weighted_ensemble = True,excluded_model_types=excluded_model_types,time_limit=3600*11//2)

No path specified. Models will be saved in: "AutogluonModels/ag-20230701_185451/"
Presets specified: ['experimental_best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 19800s
AutoGluon will save models to "AutogluonModels/ag-20230701_185451/Predictor_EC1/"
AutoGluon Version:  0.8.2
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 27 10:54:51 UTC 2023
Disk Space Avail:   20.94 GB / 20.96 GB (99.9%)
Train Data Rows:    15838
Train Data Columns: 43
Label Column: EC1
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    32333.97 MB
	Train Data (Original)  Memory Usage: 5.12 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Se

Fitting TabularPredictor for label: EC1 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 40 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  3 | ['NumHeteroatoms', 'fr_COO', 'fr_COO2']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 40 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  3 | ['NumHeteroatoms', 'fr_COO', 'fr_COO2']
	0.2s = Fit runtime
	43 features in original data used to generate 43 features in processed data.
	Train Data (Processed) Memory Usage: 5.12 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.26s ...
AutoGluon will gauge predictive performance using evaluation metric: 'roc_auc'
	This metric expects predicted probabilities rather than predicted class labels, so you'll need to use predict_proba() instead of predict()
	To change this, specify the eval_metric parameter of Predictor()
User-specified model hyperparameters to be fit:
{
	'FT_TRANSFORMER': {},
	

Fitting TabularPredictor for label: EC2 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 40 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  4 | ['NumHeteroatoms', 'fr_COO', 'fr_COO2', 'EC1']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 40 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])       :  3 | ['NumHeteroatoms', 'fr_COO', 'fr_COO2']
		('int', ['bool']) :  1 | ['EC1']
	0.2s = Fit runtime
	44 features in original data used to generate 44 features in processed data.
	Train Data (Processed) Memory Usage: 5.13 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.27s ...
AutoGluon will gauge predictive performance using evaluation metric: 'roc_auc'
	This metric expects predicted probabilities rather than predicted class labels, so you'll need to use predict_proba() instead of predict()
	To change this, specify the eval_metric parameter of Predictor()
User-specified model hyp

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('AutogluonModels/ag-20230701_185451/')


In [22]:
# predictions = multi_predictor.predict(test)
# predictions

In [23]:
prediction_prob = multi_predictor.predict_proba(test)
prediction_prob

Predicting with TabularPredictor for label: EC1 ...


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Predicting with TabularPredictor for label: EC2 ...


Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F4/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F2/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F3/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F1/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F5/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/FTTransformer_BAG_L1/S1F8/automm_model/model.ckpt
Load pretrained checkpoint: /kaggle/working/AutogluonModels/ag-20230701_185451/Predictor_EC2/models/

{'EC1':              0         1
 0     0.598200  0.401800
 1     0.176770  0.823230
 2     0.223024  0.776976
 3     0.258610  0.741390
 4     0.204256  0.795744
 ...        ...       ...
 9888  0.381630  0.618370
 9889  0.195214  0.804786
 9890  0.616115  0.383885
 9891  0.619767  0.380233
 9892  0.636340  0.363660
 
 [9893 rows x 2 columns],
 'EC2':              0         1
 0     0.150858  0.849142
 1     0.179846  0.820154
 2     0.263520  0.736480
 3     0.210335  0.789665
 4     0.266886  0.733114
 ...        ...       ...
 9888  0.261083  0.738917
 9889  0.158418  0.841582
 9890  0.117403  0.882597
 9891  0.097946  0.902054
 9892  0.085372  0.914628
 
 [9893 rows x 2 columns]}

In [24]:
submission

,id,EC1,EC2
0,14838,0.5,0.5
1,14839,0.5,0.5
2,14840,0.5,0.5
3,14841,0.5,0.5
4,14842,0.5,0.5
...,...,...,...
9888,24726,0.5,0.5
9889,24727,0.5,0.5
9890,24728,0.5,0.5
9891,24729,0.5,0.5


In [25]:
submission['EC1']=prediction_prob['EC1'][1]
submission['EC2']=prediction_prob['EC2'][1]

In [26]:
submission

,id,EC1,EC2
0,14838,0.401800,0.849142
1,14839,0.823230,0.820154
2,14840,0.776976,0.736480
3,14841,0.741390,0.789665
4,14842,0.795744,0.733114
...,...,...,...
9888,24726,0.618370,0.738917
9889,24727,0.804786,0.841582
9890,24728,0.383885,0.882597
9891,24729,0.380233,0.902054


In [27]:
!rm -rf /kaggle/working/*

In [28]:
submission.to_csv('/kaggle/working/submission.csv', index=False)